In [33]:
#change to location of root directory
root = "/hdd/"
folders = ["pringles_1k_1554610789151371002",
           "black_decker_1k_1554675460418814897",
          "lego_toy_1k_1554674193206299066",
          "pringles_1k_1554677386797606945"]


In [34]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch
from PIL import Image
import numpy as np
class ourDataset(Dataset):
    def __init__(self, folder):
        
        labelFileName = folder + "/labels.txt"
        file = open(labelFileName,"r")
        
        lines = file.readlines()
        self.count = len(lines)
        self.examples = []
        for line in lines:
            array = line.split(" ")
            example = [array[0],array[1],
                       int(array[2]),int(array[3]),
                       int(array[4]),int(array[5])]
            #print(example)
            self.examples.append(example)
            
        self.image_folder = folder + "/images/"
        self.transformations = \
            transforms.Compose([transforms.ToTensor()])
        
    def __getitem__(self, index):
        # stuff
        
        single_image_name = self.image_folder +self.examples[index][0]
        img_as_img = Image.open(single_image_name)
        img = self.transformations(img_as_img) 
        label = 1
        
        
        return (img, label)

    def __len__(self):
        return self.count # of how many data(images?) you have

In [35]:
batch_size = 16
validation_split = .2
random_seed= 42

In [37]:
from torch.utils.data.sampler import SubsetRandomSampler

datasets = []
for f in folders:
    folder = root + f
    datasets.append(ourDataset(folder))
dataset = torch.utils.data.ConcatDataset(datasets)

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
